<font size = "6"> Fordham Sports Analytics Society Big Data Bowl 2023 - Model Creation </font>

<font size = "4"> Build regression model to determine the distance of an individual rusher at the end of QB "Possession". </font>

- Authors:  Peter Majors, Chris Orlando, Jack Townsend, and Etienne Busnel
- Kaggle:  https://www.kaggle.com/competitions/nfl-big-data-bowl-2023/overview (Resources)
- Our Github:  https://github.com/peterlmajors/FSAS_BigDataBowl_2023 (Up-To-Date Code)

In [2]:
#Import Relevant Packages

#Data Manipulation
import pandas as pd
import numpy as np
import math

#Notebook Settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

#Data Viz
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('ggplot')
import seaborn as sns 
import plotly.graph_objects as go

#Machine Learning and Statistics
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# import statsmodels.formula.api as smf
# import xgboost as xgb

np.random.seed(0)

In [3]:
#Import Model Ready Data
df = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/df_model.csv")

In [4]:
#Show The Data
df.head()

,Unnamed: 0,displayName,game_play_nfl_Id,pff_positionLinedUp,ff_in_imm_zone,ff_out_imm_zone,s_rusher_ff_imm_box,a_rusher_ff_imm_box,rusher_dist_from_qb_diff_beg_end_imm_zone,diff_btw_rusher_dir_blocker_o_ff_in_imm,blocker_o_diff_beg_end_imm_zone,time_rusher_in_imm_zone,time_in_imm_zone_after_out,pff_blockType,defendersInBox,qb_pressure_sack_percent,rusher_avg_dist_from_qb_end_poss,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,nflId_blocker,nflId_rusher,displayName_rusher,pff_positionLinedUp_rusher,QB_on_play,team
0,0,Ryan Jensen,2021090900-97-40151,C,10.0,15.0,1.99,1.41,0.059230,39.22,27.07,5.0,25.0,SW,6.0,15.4,5.668404,0.0,0.0,0.0,0.0,40151.0,44955.0,Carlos Watkins,DRT,Tom Brady,TB
1,1,Donovan Smith,2021090900-97-42377,LT,23.0,38.0,4.87,2.94,1.476465,278.42,25.55,15.0,2.0,PP,6.0,15.4,4.330691,0.0,0.0,1.0,0.0,42377.0,42403.0,Randy Gregory,ROLB,Tom Brady,TB
2,2,Ali Marpet,2021090900-97-42404,LG,6.0,33.0,0.56,2.42,-1.937609,1.34,39.02,27.0,7.0,PP,6.0,15.4,5.668404,0.0,0.0,0.0,0.0,42404.0,44955.0,Carlos Watkins,DRT,Tom Brady,TB
3,3,Alex Cappa,2021090900-97-46163,RG,12.0,31.0,3.57,3.66,2.656336,0.64,21.07,19.0,9.0,SW,6.0,15.4,4.675686,0.0,0.0,1.0,0.0,46163.0,53441.0,Micah Parsons,LILB,Tom Brady,TB
4,4,Tristan Wirfs,2021090900-97-52421,RT,26.0,40.0,2.44,2.07,1.806022,94.26,165.64,14.0,NaN,PP,6.0,15.4,4.675686,0.0,0.0,1.0,0.0,52421.0,53441.0,Micah Parsons,LILB,Tom Brady,TB


<font size = "5"> Data Preprocessing </font>
